In [1]:
import pandas as pd
import numpy as np

test_ad=pd.read_csv("../test/ad.csv")
test_click_log=pd.read_csv("../test/click_log.csv")

train_merged=pd.read_csv("./train_merged.csv")
display(train_merged)

features=["creative_id","ad_id","product_id","product_category","advertiser_id","industry","time","click_times"]
targets=["age","gender"]

In [2]:
# tmpdf=train_merged.sample(frac=0.1)
# tmpdf.info()
# display(tmpdf)
# tmpdf.describe()

# %%capture cap --no-stderr
import lightgbm as lgb
from sklearn.model_selection import train_test_split
features=["creative_id","ad_id","product_id","product_category","advertiser_id","industry","time","click_times"]

df=train_merged.drop(columns=["age","user_id"])
# convert {1,2} to binary value {0,1}
df["gender"]-=1
# split train data into 2 parts (cross validation)
train=df.sample(frac=0.8) #random state is a seed value
test=df.drop(train.index)
y_train=train.pop("gender")
X_train=train
y_test=test.pop("gender")
X_test=test
# display(X_train,y_train)
# display(X_test,y_test)


train_data = lgb.Dataset(X_train,y_train)
train_data.save_binary('train.bin')


In [ ]:
# %%capture cap --no-stderr
%reset -f
import lightgbm as lgb
train_data = lgb.Dataset('gender.bin')
lgb_params = {'num_leaves': 2**6-1,
              'min_data_in_leaf': 25, 
              'objective':'binary',
              'max_depth': -1,
              'learning_rate': 0.1,
              'boosting': 'gbdt',
              'feature_fraction': 0.6,
              'bagging_fraction': 0.9,
              'bagging_seed': 11,
              'metric': 'auc',
#               'seed':1024,
              'nthread':12,
             }
with open('lgb.md', 'a') as f:
    f.write(str(lgb_params))
print("start training")
cv_results = lgb.cv(
    lgb_params, train_data, num_boost_round=1000, nfold=5, stratified=False, shuffle=True,
    early_stopping_rounds=50, verbose_eval=1, show_stdv=True, seed=0)
# print('best num_boost_round:', len(cv_results['auc-mean']))
# print('best cv score:', cv_results['auc-mean'])

start training
[1]	cv_agg's auc: 0.655103 + 0.000246757
[2]	cv_agg's auc: 0.664998 + 0.000205661
[3]	cv_agg's auc: 0.668317 + 6.79261e-05
[4]	cv_agg's auc: 0.66964 + 0.000355484
[5]	cv_agg's auc: 0.670757 + 0.000544024
[6]	cv_agg's auc: 0.673907 + 0.000377497
[7]	cv_agg's auc: 0.675651 + 0.000551716
[8]	cv_agg's auc: 0.677268 + 0.000640823
[9]	cv_agg's auc: 0.678158 + 0.000899701
[10]	cv_agg's auc: 0.679363 + 0.000781556
[11]	cv_agg's auc: 0.680395 + 0.000851703
[12]	cv_agg's auc: 0.681316 + 0.00094943
[13]	cv_agg's auc: 0.682439 + 0.000999857
[14]	cv_agg's auc: 0.683199 + 0.000862841
[15]	cv_agg's auc: 0.683822 + 0.000791141
[16]	cv_agg's auc: 0.684391 + 0.000820235
[17]	cv_agg's auc: 0.685201 + 0.000668377
[18]	cv_agg's auc: 0.685958 + 0.000807344
[19]	cv_agg's auc: 0.686751 + 0.000675411
[20]	cv_agg's auc: 0.687396 + 0.000519532
[21]	cv_agg's auc: 0.687939 + 0.000486729
[22]	cv_agg's auc: 0.688583 + 0.000435666
[23]	cv_agg's auc: 0.689327 + 0.000382542
[24]	cv_agg's auc: 0.689912 + 

with open('lgb.txt', 'w') as f:
    f.write(cap.stdout)

# !pip install lightgbm
import lightgbm as lgb
from sklearn.model_selection import train_test_split
features=["creative_id","ad_id","product_id","product_category","advertiser_id","industry","time","click_times"]

df=train_merged.drop(columns=["gender","user_id"])
# split train data into 2 parts (cross validation)
train=df.sample(frac=0.8) #random state is a seed value
test=df.drop(train.index)
y_train=train.pop("age")
X_train=train
y_test=test.pop("age")
X_test=test
# display(X_train,y_train)
# display(X_test,y_test)
train_data = lgb.Dataset(X_train,y_train)
train_data.save_binary('age.bin')

In [ ]:
%reset -f
import lightgbm as lgb
train_data = lgb.Dataset('age.bin')
params = {'num_leaves': 2**6-1,
          'min_data_in_leaf': 25, 
          'objective':'regression_l2',
          'max_depth': -1,
          'learning_rate': 0.1,
          'min_child_samples': 20,
          'boosting': 'gbdt',
          'feature_fraction': 0.6,
          'bagging_fraction': 0.9,
          'bagging_seed': 11,
          'metric': 'mae',
          'seed':1024,
          'lambda_l1': 0.2,
          'nthread':12,
         }
print("start training")
cv_results = lgb.cv(
    params, train_data, num_boost_round=1000, nfold=5, stratified=False, shuffle=True,
    early_stopping_rounds=50, verbose_eval=1, show_stdv=True, seed=0)
print('best num_boost_round:', len(cv_results['mae-mean']))
print('best cv score:', cv_results['mae-mean'])
# bst = lgb.train(param, train_data, num_round,categorical_feature=features)
# bst.save_model('test_model.txt')

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train,free_raw_data=False)
params = {
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric': 'auc',
          'nthread':12,
          'learning_rate':0.1
          }
 
### 交叉验证(调参)
print('交叉验证')
max_auc = float('0')
best_params = {}
 
# 准确率
print("调参1：提高准确率")
for num_leaves in range(5,100,5):
    for max_depth in range(3,8,1):
        params['num_leaves'] = num_leaves
        params['max_depth'] = max_depth
 
        cv_results = lgb.cv(
                            params,
                            lgb_train,
                            seed=1,
                            nfold=5,
                            metrics=['auc'],
                            early_stopping_rounds=10,
                            verbose_eval=True
                            )
            
        mean_auc = pd.Series(cv_results['auc-mean']).max()
        boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
            
        if mean_auc >= max_auc:
            max_auc = mean_auc
            best_params['num_leaves'] = num_leaves
            best_params['max_depth'] = max_depth
if 'num_leaves' and 'max_depth' in best_params.keys():          
    params['num_leaves'] = best_params['num_leaves']
    params['max_depth'] = best_params['max_depth']
 
# 过拟合
print("调参2：降低过拟合")
for max_bin in range(5,256,10):
    for min_data_in_leaf in range(1,102,10):
            params['max_bin'] = max_bin
            params['min_data_in_leaf'] = min_data_in_leaf
            
            cv_results = lgb.cv(
                                params,
                                lgb_train,
                                seed=1,
                                nfold=5,
                                metrics=['auc'],
                                early_stopping_rounds=10,
                                verbose_eval=True
                                )
                    
            mean_auc = pd.Series(cv_results['auc-mean']).max()
            boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
 
            if mean_auc >= max_auc:
                max_auc = mean_auc
                best_params['max_bin']= max_bin
                best_params['min_data_in_leaf'] = min_data_in_leaf
if 'max_bin' and 'min_data_in_leaf' in best_params.keys():
    params['min_data_in_leaf'] = best_params['min_data_in_leaf']
    params['max_bin'] = best_params['max_bin']
 
print("调参3：降低过拟合")
for feature_fraction in [0.6,0.7,0.8,0.9,1.0]:
    for bagging_fraction in [0.6,0.7,0.8,0.9,1.0]:
        for bagging_freq in range(0,50,5):
            params['feature_fraction'] = feature_fraction
            params['bagging_fraction'] = bagging_fraction
            params['bagging_freq'] = bagging_freq
            
            cv_results = lgb.cv(
                                params,
                                lgb_train,
                                seed=1,
                                nfold=5,
                                metrics=['auc'],
                                early_stopping_rounds=10,
                                verbose_eval=True
                                )
                    
            mean_auc = pd.Series(cv_results['auc-mean']).max()
            boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
 
            if mean_auc >= max_auc:
                max_auc=mean_auc
                best_params['feature_fraction'] = feature_fraction
                best_params['bagging_fraction'] = bagging_fraction
                best_params['bagging_freq'] = bagging_freq
 
if 'feature_fraction' and 'bagging_fraction' and 'bagging_freq' in best_params.keys():
    params['feature_fraction'] = best_params['feature_fraction']
    params['bagging_fraction'] = best_params['bagging_fraction']
    params['bagging_freq'] = best_params['bagging_freq']
 
 
print("调参4：降低过拟合")
for lambda_l1 in [1e-5,1e-3,1e-1,0.0,0.1,0.3,0.5,0.7,0.9,1.0]:
    for lambda_l2 in [1e-5,1e-3,1e-1,0.0,0.1,0.4,0.6,0.7,0.9,1.0]:
        params['lambda_l1'] = lambda_l1
        params['lambda_l2'] = lambda_l2
        cv_results = lgb.cv(
                            params,
                            lgb_train,
                            seed=1,
                            nfold=5,
                            metrics=['auc'],
                            early_stopping_rounds=10,
                            verbose_eval=True
                            )
                
        mean_auc = pd.Series(cv_results['auc-mean']).max()
        boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
 
        if mean_auc >= max_auc:
            max_auc=mean_auc
            best_params['lambda_l1'] = lambda_l1
            best_params['lambda_l2'] = lambda_l2
if 'lambda_l1' and 'lambda_l2' in best_params.keys():
    params['lambda_l1'] = best_params['lambda_l1']
    params['lambda_l2'] = best_params['lambda_l2']
 
print("调参5：降低过拟合2")
for min_split_gain in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    params['min_split_gain'] = min_split_gain
    
    cv_results = lgb.cv(
                        params,
                        lgb_train,
                        seed=1,
                        nfold=5,
                        metrics=['auc'],
                        early_stopping_rounds=10,
                        verbose_eval=True
                        )
            
    mean_auc = pd.Series(cv_results['auc-mean']).max()
    boost_rounds = pd.Series(cv_results['auc-mean']).idxmax()
 
    if mean_auc >= max_auc:
        max_auc=mean_auc
        
        best_params['min_split_gain'] = min_split_gain
if 'min_split_gain' in best_params.keys():
    params['min_split_gain'] = best_params['min_split_gain']
 
print(best_params)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
df=tmpdf
features=["creative_id"]
v = TfidfVectorizer(max_df=1.0,min_df=1,lowercase=False,tokenizer=lambda x:x)
for feature in features:
    cur=df.groupby("user_id")[feature].apply(list)
    v_fit = v.fit_transform(cur)
    word=v.get_feature_names()
    display(v_fit)
    display(type(v_fit))
#     for i in range(len(weight)):
#         print("-----output feature={}, user_id={} tf-idf-----".format(feature,i+1))
#         for j in range(len(word)):
#             print(word[j],weight[i][j])